In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data146007  dataset  readme.json


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/57/f4/a69c20ee4f660081a7dedb1ac57f29be9378e04edfcb90c526b923d4bebc/beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/49/37/673d6490efc51ec46d198c75903d99de59baffdd47aea3d071b80a9e4e89/soupsieve-2.4.1-py3-none-any.whl (36 kB)

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

## 基于PaddlePaddle2.0-构建卷积网络模型LeNet-5

### 1. LeNet-5模型表达式

LeNet-5是卷积神经网络模型的早期代表，它由LeCun在1998年提出。该模型采用顺序结构，主要包括7层（2个卷积层、2个池化层和3个全连接层），卷积层和池化层交替排列。以mnist手写数字分类为例构建一个LeNet-5模型,每个手写数字图片样本的宽与高均为28像素，样本标签值是0~9，代表0至9十个数字。

![](https://ai-studio-static-online.cdn.bcebos.com/c758063e28754e20ac3ec70cef5ca1b0168ad923000d47f1bd686b59d2f3c23b)

图1. 单样本视角的LeNet-5模型原理

下面详细解析LeNet-5模型的正向传播过程。

（1）卷积层L1

单样本视角。L1层的输入数据形状大小为$\mathbb{R}^{1 \times 28 \times 28}$，表示通道数量为1，行与列的大小都为28。输出数据形状大小为$\mathbb{R}^{6 \times 24 \times 24}$，表示通道数量为6，行与列维都为24。

批量样本视角。设批量大小为m。L1层的输入数据形状大小为$\mathbb{R}^{m \times 1 \times 28 \times 28}$，表示样本批量为m，通道数量为1，行与列的大小都为28。L1层的输出数据形状大小为$\mathbb{R}^{m \times 6 \times 24 \times 24}$，表示样本批量为m，通道数量为6，行与列维都为24。

参数视角。L1层的权重形状大小$\mathbb{R}^{6 \times 1 \times 5 \times 5}$为，偏置项形状大小为6。

这里有两个问题很关键：一是，为什么通道数从1变成了6呢？原因是模型的卷积层L1设定了6个卷积核，每个卷积核都与输入数据发生运算，最终分别得到6组数据。二是，为什么行列大小从28变成了24呢？原因是每个卷积核的行维与列维都为5，卷积核（5×5）在输入数据（28×28）上移动，且每次移动步长为1，那么输出数据的行列大小分别为28-5+1=24。

（2）池化层L2

从单样本视角。L2层的输入数据大小要和L1层的输出数据大小保持一致。输入数据形状大小为$\mathbb{R}^{6 \times 24 \times 24}$，表示通道数量为6，行与列的大小都为24。L2层的输出数据形状大小为$\mathbb{R}^{6 \times 12 \times 12}$，表示通道数量为6，行与列维都为12。

从批量样本视角。设批量大小为m。L2层的输入数据形状大小为$\mathbb{R}^{m \times 6 \times 24 \times 24}$，表示样本批量为m，通道数量为6，行与列的大小都为24。L2层的输出数据形状大小为$\mathbb{R}^{m \times 6 \times 12 \times 12}$，表示样本批量为m，通道数量为6，行与列维都为12。为什么行列大小从24变成了12呢？原因是池化层中的过滤器形状大小为2×2，其在输入数据（24×24）上移动，且每次移动步长（跨距）为2，每次选择4个数（2×2）中最大值作为输出，那么输出数据的行列大小分别为24÷2=12。

（3）卷积层L3

单样本视角。L3层的输入数据形状大小为$\mathbb{R}^{6 \times 12 \times 12}$，表示通道数量为6，行与列的大小都为12。L3层的输出数据形状大小为$\mathbb{R}^{6 \times 8 \times 8}$，表示通道数量为16，行与列维都为8。

批量样本视角。设批量大小为m。L3层的输入数据形状大小为$\mathbb{R}^{m \times 6 \times 12 \times 12}$，表示样本批量为m，通道数量为6，行与列的大小都为12。L3层的输出数据形状大小为$\mathbb{R}^{m \times 16 \times 8 \times 8}$，表示样本批量为m，通道数量为16，行与列维都为8。

参数视角。L3层的权重形状大小为$\mathbb{R}^{m \times 16 \times 6 \times 5 \times 5}$，偏置项形状大小为16。

（4）池化层L4

从单样本视角。L4层的输入数据形状大小与L3层的输出数据大小一致。L4层的输入数据形状大小为$\mathbb{R}^{16 \times 8 \times 8}$，表示通道数量为16，行与列的大小都为8。L4层的输出数据形状大小为$\mathbb{R}^{16 \times 4 \times 4}$，表示通道数量为16，行与列维都为4。

从批量样本视角。设批量大小为m。L4层的输入数据形状大小为$\mathbb{R}^{m \times 16 \times 8 \times 8}$，表示样本批量为m，通道数量为16，行与列的大小都为8。L4层的输出数据形状大小为$\mathbb{R}^{m \times 16 \times 4 \times 4}$，表示样本批量为m，通道数量为16，行与列维都为4。池化层L4中的过滤器形状大小为2×2，其在输入数据（形状大小24×24）上移动，且每次移动步长（跨距）为2，每次选择4个数（形状大小2×2）中最大值作为输出。

（5）线性层L5

从单样本视角。由于L5层是线性层，其输入大小为一维，所以需要把L4层的输出数据大小进行重新划分。L4层的输出形状大小为$\mathbb{R}^{16 \times 4 \times 4}$，则L5层的一维输入形状大小为16×4×4=256。L4层的一维输出大小为120。

从批量样本视角。设批量大小为m。L5层输入数据形状大小为$\mathbb{R}^{m \times 256}$，表示样本批量为m，输入特征数量为256。输出数据形状大小为$\mathbb{R}^{m \times 120}$，表示样本批量为m，输出特征数量为120。

（6）线性层L6

从单样本视角。L6层的输入特征数量为120。L6层的输出特征数量为84。

从批量样本视角。设批量大小为m。L6层的输入数据形状大小为$\mathbb{R}^{m \times 120}$，表示样本批量为m，输入特征数量为120。L6层的输出数据形状大小为$\mathbb{R}^{m \times 84}$，表示样本批量为m，输出特征数量为84。

（7）线性层L7

从单样本视角。L7层的输入特征数量为84。L7层的输出特征数量为10。

从批量样本视角。设批量大小为m。L7层的输入数据形状大小为$\mathbb{R}^{m \times 84}$，表示样本批量为m，输入特征数量为84。L7层的输出数据形状大小为$\mathbb{R}^{m \times 10}$，表示样本批量为m，输出特征数量为10。

由于是分类问题，我们选择交叉熵损失函数。交叉熵主要用于衡量估计值与真实值之间的差距。交叉熵值越小，模型预测效果越好。

$E(\mathbf{y}^{i},\mathbf{\hat{y}}^{i})=-\sum_{j=1}^{q}\mathbf{y}_{j}^{i}ln(\mathbf{\hat{y}}_{j}^{i})$

其中，$\mathbf{y}^{i} \in \mathbb{R}^{q}$为真实值，$y_{j}^{i}$是$\mathbf{y}^{i}$中的元素(取值为0或1)，$j=1,...,q$。$\mathbf{\hat{y}^{i}} \in \mathbb{R}^{q}$是预测值（样本在每个类别上的概率）。

定义好了正向传播过程之后，接着随机化初始参数，然后便可以计算出每层的结果，每次将得到m×10的矩阵作为预测结果，其中m是小批量样本数。接下来进行反向传播过程，预测结果与真实结果之间肯定存在差异，以缩减该差异作为目标，计算模型参数梯度。进行多轮迭代，便可以优化模型，使得预测结果与真实结果之间更加接近。

作业要求：

参考lenet模型，构建一个用于宝石识别的cnn模型，训练模型力求精确度越高越好

In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data146007  dataset  readme.json


In [ ]:
#导入需要的包
import os
import zipfile
import random
import json
import cv2
import numpy as np
from PIL import Image
import paddle
import matplotlib.pyplot as plt
from paddle.io import Dataset

# 1. 数据准备

In [ ]:
'''
参数配置
'''
train_parameters = {
    "input_size": [16,224,224],                           #输入图片的shape
    "class_dim": -1,                                     #分类数
    "src_path":"data/data146007/archive_train.zip",       #原始数据集路径
    "target_path":"/home/aistudio/data/dataset",        #要解压的路径 
    "train_list_path": "./train.txt",              #train_data.txt路径
    "eval_list_path": "./eval.txt",                  #eval_data.txt路径
    "label_list_path": "./label.txt",                  #eval_data.txt路径
    "label_dict":{},                                    #标签字典
    "readme_path": "/home/aistudio/data/readme.json",   #readme.json路径
    "num_epochs": 40,                                    #训练轮数
    "train_batch_size": 16,                             #批次的大小
    "learning_strategy": {                              #优化函数相关的配置
        "lr": 0.001                                     #超参数学习率
    } 
}



In [ ]:
print(train_parameters['learning_strategy']['lr'])

0.001


In [ ]:
def unzip_data(src_path,target_path):

    '''
    解压原始数据集，将src_path路径下的zip包解压至data/dataset目录下
    '''

    if(not os.path.isdir(target_path)):    
        z = zipfile.ZipFile(src_path, 'r')
        z.extractall(path=target_path)
        z.close()
    else:
        print("文件已解压")


In [ ]:
def get_data_list(target_path,train_list_path,eval_list_path,label_list_path):
    '''
    生成数据列表
    '''
    #存放所有类别的信息
    class_detail = []
    #获取所有类别保存的文件夹名称
    data_list_path=target_path
    class_dirs = os.listdir(data_list_path)
    if '__MACOSX' in class_dirs:
        class_dirs.remove('__MACOSX')
    # #总的图像数量
    all_class_images = 0
    # #存放类别标签
    class_label=0
    # #存放类别数目
    class_dim = 0
    # #存储要写进eval.txt和train.txt中的内容
    trainer_list=[]
    eval_list=[]
    label_list=[]
    #读取每个类别
    for class_dir in class_dirs:
        if class_dir != ".DS_Store":
            if ' ' in class_dir:    #若文件夹名中存在空格
                d=class_dir.split(' ')    #以空格作为分隔符
                new_name1=os.path.join(d[0]+"_"+d[1])
                os.rename(os.path.join(data_list_path,class_dir),os.path.join(data_list_path,new_name1))  #重命名 
                class_dir=new_name1           
            class_dim += 1
            #每个类别的信息
            class_detail_list = {}
            eval_sum = 0
            trainer_sum = 0
            #统计每个类别有多少张图片
            class_sum = 0
            #获取类别路径 
            path = os.path.join(data_list_path,class_dir)
            # 获取所有图片
            img_paths = os.listdir(path)
            for img_path in img_paths:                                  # 遍历文件夹下的每个图片
                if img_path =='.DS_Store':
                    continue
                if ' ' in img_path:    #若文件夹名中存在空格
                    d=img_path.split(' ')    #以空格作为分隔符
                    new_name2=os.path.join(d[0]+"_"+d[1])
                    os.rename(os.path.join(path,img_path),os.path.join(path,new_name2))  #重命名
                    img_path=new_name2
                name_path = os.path.join(path,img_path)                       # 每张图片的路径
                if class_sum % 15 == 0:                                 # 每10张图片取一个做验证数据
                    eval_sum += 1                                       # eval_sum为测试数据的数目
                    eval_list.append(name_path + "\t%d" % class_label + "\n")
                else:
                    trainer_sum += 1 
                    trainer_list.append(name_path + "\t%d" % class_label + "\n")#trainer_sum测试数据的数目
                class_sum += 1                                          #每类图片的数目
                all_class_images += 1                                   #所有类图片的数目
            
            # 说明的json文件的class_detail数据
            class_detail_list['class_name'] = class_dir             #类别名称
            class_detail_list['class_label'] = class_label          #类别标签
            class_detail_list['class_eval_images'] = eval_sum       #该类数据的测试集数目
            class_detail_list['class_trainer_images'] = trainer_sum #该类数据的训练集数目
            class_detail.append(class_detail_list)  
            #初始化标签列表
            train_parameters['label_dict'][str(class_label)] = class_dir
            #label_dict字典中，类别标签为class_
            class_label += 1
            label_list.append(class_dir+"\n")
            
    #初始化分类数
    train_parameters['class_dim'] = class_dim
    print(train_parameters)
    #乱序  
    random.shuffle(eval_list)
    with open(eval_list_path, 'a') as f:
        for eval_image in eval_list:
            f.write(eval_image) 
    #乱序        
    random.shuffle(trainer_list) 
    with open(train_list_path, 'a') as f2:
        for train_image in trainer_list:
            f2.write(train_image) 
    #乱序        
    random.shuffle(label_list) 
    with open(label_list_path, 'a') as f2:
        for image_label in label_list:
            f2.write(image_label) 
    # 说明的json文件信息
    readjson = {}
    readjson['all_class_name'] = data_list_path                  #文件父目录
    readjson['all_class_images'] = all_class_images
    readjson['class_detail'] = class_detail
    jsons = json.dumps(readjson, sort_keys=True, indent=4, separators=(',', ': '))
    with open(train_parameters['readme_path'],'w') as f:
        f.write(jsons)
    print ('生成数据列表完成！')

In [ ]:
'''
参数初始化
'''
src_path=train_parameters['src_path']
target_path=train_parameters['target_path']
train_list_path=train_parameters['train_list_path']
eval_list_path=train_parameters['eval_list_path']
label_list_path=train_parameters['label_list_path']
batch_size=train_parameters['train_batch_size']
'''
解压原始数据到指定路径
'''
unzip_data(src_path,target_path)

'''
划分训练集与验证集，乱序，生成数据列表
'''
#每次生成数据列表前，首先清空train.txt和eval.txt
with open(train_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
with open(eval_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
    
#生成数据列表   
get_data_list(target_path,train_list_path,eval_list_path,label_list_path)

文件已解压
{'input_size': [16, 224, 224], 'class_dim': 25, 'src_path': 'data/data146007/archive_train.zip', 'target_path': '/home/aistudio/data/dataset', 'train_list_path': './train.txt', 'eval_list_path': './eval.txt', 'label_list_path': './label.txt', 'label_dict': {'0': 'Iolite', '1': 'Malachite', '2': 'Danburite', '3': 'Hessonite', '4': 'Alexandrite', '5': 'Fluorite', '6': 'Almandine', '7': 'Cats_Eye', '8': 'Diamond', '9': 'Emerald', '10': 'Quartz_Beer', '11': 'Tanzanite', '12': 'Benitoite', '13': 'Onyx_Black', '14': 'Garnet_Red', '15': 'Jade', '16': 'Variscite', '17': 'Zircon', '18': 'Kunzite', '19': 'Pearl', '20': 'Sapphire_Blue', '21': 'Beryl_Golden', '22': 'Labradorite', '23': 'Carnelian', '24': 'Rhodochrosite'}, 'readme_path': '/home/aistudio/data/readme.json', 'num_epochs': 40, 'train_batch_size': 16, 'learning_strategy': {'lr': 0.001}}
生成数据列表完成！


In [ ]:
from paddle.vision import transforms as T

class Reader(Dataset):
    def __init__(self, data_path, mode='train'):
        """
        数据读取器
        :param data_path: 数据集所在路径
        :param mode: train or eval
        """
        super().__init__()
        self.data_path = data_path
        self.img_paths = []
        self.labels = []
        #数据增强
        self.transform =T.Compose([T.Resize((256, 256)), 
                        T.CenterCrop(224), 
                        T.RandomHorizontalFlip(0.5),
                        T.RandomRotation(15),
                        T.Transpose(),
                        T.Normalize(mean=255, data_format='CHW', to_rgb=True),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])
        if mode == 'train':
            with open(os.path.join(self.data_path, "train.txt"), "r", encoding="utf-8") as f:
                self.info = f.readlines()
            for img_info in self.info:
                img_path, label = img_info.strip().split('\t')
                self.img_paths.append(img_path)
                self.labels.append(int(label))

        else:
            with open(os.path.join(self.data_path, "eval.txt"), "r", encoding="utf-8") as f:
                self.info = f.readlines()
            for img_info in self.info:
                img_path, label = img_info.strip().split('\t')
                self.img_paths.append(img_path)
                self.labels.append(int(label))


    def __getitem__(self, index):
        """
        获取一组数据
        :param index: 文件索引号
        :return:
        """
        # 第一步打开图像文件并获取label值
        img_path = self.img_paths[index]
        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB') 
        img = img.resize((224, 224), Image.BILINEAR)
        img = np.array(img).astype('float32')
        img = img.transpose((2, 0, 1)) / 255
        label = self.labels[index]
        label = np.array([label], dtype="int64")
        return img, label

    def print_sample(self, index: int = 0):
        print("文件名", self.img_paths[index], "\t标签值", self.labels[index])

    def __len__(self):
        return len(self.img_paths)

In [ ]:
train_dataset = Reader('/home/aistudio/',mode='train')

eval_dataset = Reader('/home/aistudio/',mode='eval')
#训练数据加载
train_loader = paddle.io.DataLoader(train_dataset, batch_size=16, shuffle=True)
#测试数据加载
eval_loader = paddle.io.DataLoader(eval_dataset, batch_size = 8, shuffle=False)

In [ ]:
train_dataset.print_sample(200)
print(train_dataset.__len__())
eval_dataset.print_sample(0)
print(eval_dataset.__len__())
print(eval_dataset.__getitem__(10)[0].shape)
print(eval_dataset.__getitem__(10)[1].shape)

文件名 /home/aistudio/data/dataset/Labradorite/labradorite_35.jpg 	标签值 22
742
文件名 /home/aistudio/data/dataset/Malachite/malachite_18.jpg 	标签值 1
69
(3, 224, 224)
(1,)


In [ ]:
!pip install paddlex==2.0.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import paddlex as pdx
from paddlex import transforms as T
train_transforms = T.Compose(
    [T.RandomCrop(crop_size=224), T.RandomHorizontalFlip(), T.Normalize()])

eval_transforms = T.Compose([
    T.ResizeByShort(short_size=256), T.CenterCrop(crop_size=224), T.Normalize()
])

In [45]:
train_dataset = pdx.datasets.ImageNet(
    data_dir='/home/aistudio/data/dataset',
    file_list='/home/aistudio/train.txt',
    label_list='/home/aistudio/label.txt',
    transforms=train_transforms,
    shuffle=True)
    
eval_dataset = pdx.datasets.ImageNet(
    data_dir='/home/aistudio/data/dataset',
    file_list='/home/aistudio/eval.txt',
    label_list='/home/aistudio/label.txt',
    transforms=eval_transforms)

num_classes = len(train_dataset.labels)
model = pdx.cls.ResNet101_vd_ssld(num_classes=num_classes)
model.train(num_epochs=20,
            train_dataset=train_dataset,
            train_batch_size=16,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[6, 8],
            save_interval_epochs=1,
            learning_rate=0.005,
            save_dir='output/resNet101_vd_ssld',
            use_vdl=True)

2023-05-16 15:57:51 [INFO]	Starting to read file list from dataset...
2023-05-16 15:57:51 [INFO]	742 samples in file /home/aistudio/train.txt
2023-05-16 15:57:51 [INFO]	Starting to read file list from dataset...
2023-05-16 15:57:51 [INFO]	69 samples in file /home/aistudio/eval.txt
2023-05-16 15:57:52 [INFO]	Loading pretrained model from output/resNet101_vd_ssld/pretrain/ResNet101_vd_ssld_pretrained.pdparams


Exception in thread Thread-62:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 620, in _get_data
    data = self._data_queue.get(timeout=self._timeout)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/queues.py", line 105, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 534, in _thread_loop
    batch = self._get_data()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/

2023-05-16 15:57:54 [WARNING]	[SKIP] Shape of pretrained params out.weight doesn't match.(Pretrained: [2048, 1000], Actual: [2048, 50])
2023-05-16 15:57:54 [WARNING]	[SKIP] Shape of pretrained params out.bias doesn't match.(Pretrained: [1000], Actual: [50])
2023-05-16 15:57:54 [INFO]	There are 530/532 variables loaded into ResNet101_vd_ssld.


In [ ]:
# Batch=0
# Batchs=[]
# all_train_accs=[]
# def draw_train_acc(Batchs, train_accs):
#     title="training accs"
#     plt.title(title, fontsize=24)
#     plt.xlabel("batch", fontsize=14)
#     plt.ylabel("acc", fontsize=14)
#     plt.plot(Batchs, train_accs, color='green', label='training accs')
#     plt.legend()
#     plt.grid()
#     plt.show()

# all_train_loss=[]
# def draw_train_loss(Batchs, train_loss):
#     title="training loss"
#     plt.title(title, fontsize=24)
#     plt.xlabel("batch", fontsize=14)
#     plt.ylabel("loss", fontsize=14)
#     plt.plot(Batchs, train_loss, color='red', label='training loss')
#     plt.legend()
#     plt.grid()
#     plt.show()

# 2. 定义模型

在下方cell中自定义CNN(lenet)网络

In [ ]:
# import paddle
# from paddle.nn import Conv2D, MaxPool2D, Linear,BatchNorm2D
# import paddle.nn.functional as F
# class MyCNN(paddle.nn.Layer):  #@save
#     def __init__(self):
#         super(MyCNN, self).__init__()
#         self.conv1=Conv2D(in_channels=3, out_channels=32, kernel_size=3,stride=1,padding=2)
#         self.pool1=MaxPool2D(kernel_size=2,stride=2)
#         self.conv2=Conv2D(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=2)
#         self.pool2=MaxPool2D(kernel_size=2,stride=2)
#         # self.conv3=Conv2D(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=2)
#         # self.pool3=MaxPool2D(kernel_size=2,stride=2)
#         # if use_1x1conv:
#         #     self.conv3 = Conv2D(input_channels, num_channels,
#         #                            kernel_size=1, stride=strides)
#         # else:
#         #     self.conv3 = None
#         self.bn1 = BatchNorm2D(32)#out_channels
#         self.bn2 = BatchNorm2D(32)
#         self.fc1 = paddle.fluid.dygraph.Linear(input_dim=32*57*57, output_dim=1024, act='relu')
#         self.fc2 = paddle.fluid.dygraph.Linear(input_dim=1024, output_dim=512, act='relu')
#         self.fc3 = paddle.fluid.dygraph.Linear(input_dim=512, output_dim=128, act='relu')
#         self.fc4 = paddle.fluid.dygraph.Linear(input_dim=128, output_dim=64, act='relu')
#     def forward(self, input):
#         Y = F.relu(self.bn1(self.conv1(input)))
#         Y=self.pool1(Y)
#         Y = self.bn2(self.conv2(Y))
#         Y=self.pool2(Y)
#         # Y=self.bn2(self.conv3(Y))
#         # Y=self.pool3(Y)

#         X=F.relu(self.bn1(self.conv1(input)))
#         X=self.pool1(X)
#         X = self.bn2(self.conv2(X))
#         X=self.pool2(X)
#         # X=self.conv3(X)
#         # X=self.pool3(X)
#         # if self.conv3:
#         #     X = self.conv3(X)
#         # print(Y[0].shape)

#         Y+=X
#         Y=paddle.reshape(Y,shape=[-1,32*57*57])
#         Y=self.fc1(Y)
#         Y=self.fc2(Y)
#         Y=self.fc3(Y)
#         Y=self.fc4(Y)
#         return Y

In [ ]:
# model=MyCNN() #模型实例化
# model.train() #训练模式
# cross_entropy = paddle.nn.CrossEntropyLoss()  
# opt=paddle.optimizer.SGD(learning_rate=train_parameters['learning_strategy']['lr'], parameters=model.parameters())  #优化器设定，使用随机梯度下降算法的优化器

# epochs_num=train_parameters['num_epochs'] #迭代次数
# for pass_num in range(epochs_num):
#     for batch_id,data in enumerate(train_loader()):
#         image = data[0]
#         label = data[1]

#         predict=model(image) #数据传入model

#         loss=cross_entropy(predict,label)
#         acc=paddle.metric.accuracy(predict,label)#计算精度
        
#         if batch_id!=0 and batch_id%5==0:
#             Batch = Batch+5 
#             Batchs.append(Batch)
#             all_train_loss.append(loss.numpy()[0])
#             all_train_accs.append(acc.numpy()[0])
            
#             print("train_pass:{},batch_id:{},train_loss:{},train_acc:{}".format(pass_num,batch_id,loss.numpy(),acc.numpy()))
        
#         loss.backward() #进行反向传播求出梯度      
#         opt.step()  #执行一次优化器并进行参数更新。
#         opt.clear_grad()   #opt.clear_grad()来重置梯度

# paddle.save(model.state_dict(),'MyCNN')#保存模型

# draw_train_acc(Batchs,all_train_accs)
# draw_train_loss(Batchs,all_train_loss)

# 4. 模型评估

In [ ]:
# #模型评估
# para_state_dict = paddle.load("MyCNN")
# model = MyCNN()
# model.set_state_dict(para_state_dict) #加载模型参数
# model.eval() #验证模式

# accs = []

# for batch_id,data in enumerate(eval_loader()):#测试集
#     image=data[0]
#     label=data[1]     
#     predict=model(image)        
#     acc=paddle.metric.accuracy(predict,label)
#     accs.append(acc.numpy()[0])
#     avg_acc = np.mean(accs)
# print("当前模型在测试集上的准确率为:",avg_acc)

In [ ]:
# import os
# import zipfile

# def unzip_infer_data(src_path,target_path):
#     '''
#     解压预测数据集
#     '''
#     if(not os.path.isdir(target_path)):     
#         z = zipfile.ZipFile(src_path, 'r')
#         z.extractall(path=target_path)
#         z.close()


# def load_image(img_path):
#     '''
#     预测图片预处理
#     '''
#     img = Image.open(img_path) 
#     if img.mode != 'RGB': 
#         img = img.convert('RGB') 
#     img = img.resize((224, 224), Image.BILINEAR)
#     img = np.array(img).astype('float32') 
#     img = img.transpose((2, 0, 1))  # HWC to CHW 
#     img = img/255                # 像素值归一化 
#     return img


# infer_src_path = '/home/aistudio/data/data146007/archive_test.zip'
# infer_dst_path = '/home/aistudio/data/archive_test'
# unzip_infer_data(infer_src_path,infer_dst_path)

# 5. 模型预测

In [ ]:
'''
模型预测
'''
para_state_dict = paddle.load("MyCNN")
model = MyCNN()
model.set_state_dict(para_state_dict) #加载模型参数
model.eval() #训练模式

#展示预测图片
infer_path='data/archive_test/alexandrite_3.jpg'
img = Image.open(infer_path)
plt.imshow(img)          #根据数组绘制图像
plt.show()               #显示图像

#对预测图片进行预处理
infer_imgs = []
infer_imgs.append(load_image(infer_path))
infer_imgs = np.array(infer_imgs)

label_dic = train_parameters['label_dict']
print(label_dic)
for i in range(len(infer_imgs)):
    data = infer_imgs[i]
    dy_x_data = np.array(data).astype('float32')
    dy_x_data=dy_x_data[np.newaxis,:, : ,:]
    img = paddle.to_tensor (dy_x_data)
    out = model(img)
    lab = np.argmax(out.numpy())  #argmax():返回最大数的索引

    print("第{}个样本,被预测为：{},真实标签为：{}".format(i+1,label_dic[str(lab)],infer_path.split('/')[-1].split("_")[0]))
        
print("结束")
